## Network Science Lab 6

### Simulating the naive Network-SI model

In this lab, you will develop code to compute solutions to the naive network-SI model on a Barabasi-Albert graph. Running the cell below will generate the needed graph.

In [2]:
import networkx as nx
G = nx.barabasi_albert_graph(200,4,seed=1)
N = G.number_of_nodes()
print(G)

Graph with 200 nodes and 784 edges


The governing equations for the naive network-SI model are:

$$\frac{dx_i}{dt} =  \beta(1-x_i)\sum_{j=0}^{N-1} A_{ij} x_j, i=0,1,2,...,N-1.$$

The equations have been written with $x_i$ instead of $\left<x_i\right>$ for convenience.
The nodes are numbered from $0$ to $N-1$ which is why the sum is also from $0$ to $N-1$.
We will use the following initial condition: $x_0(t=0) = 0.001$, $x_i(t=0)=0$ for $i>0$.



1) Create a function named RHS which takes $t$, and $\textbf{x}$ as input and returns $d\textbf{x}/dt$. Note that $\textbf{x}$ and $d\textbf{x}/dt$ should each contain $N$ elements. We will use $\beta=0.2$.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
beta = 0.2 #This variable will be available within RHS below

def RHS(t, x):
    A = nx.adjacency_matrix(G).toarray()
    dxdt = beta * np.dot(A, x)
    for r in range(200):
        dxdt[r] = (1 - x[r]) * dxdt[r]
    
    return dxdt

2) Now, (a) set the variable, x0, to the initial condition, and  (b) create an array of times, t (starting at zero) at which the solution will be computed (x0 should contain $N$ elements).

In [10]:
x0 = np.zeros(200)

x0[0] = 0.001

t = np.linspace(0, 10, 1000)

3) Use solve_ivp with the BDF method to compute the solution to the naive network-SI model. Your solution should be stored in an $N \textrm{x} Nt+1$ element array (where $Nt+1$ is the size of t).

In [11]:
from scipy.integrate import solve_ivp

sol = solve_ivp(RHS, (0,10), x0, method = 'BDF')

sol

C:\Users\maxfr\AppData\Local\Temp\ipykernel_11680\1037396351.py:6: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G).toarray()


  message: 'The solver successfully reached the end of the integration interval.'
     nfev: 113
     njev: 3
      nlu: 14
      sol: None
   status: 0
  success: True
        t: array([0.00000000e+00, 9.86947787e-03, 1.97389557e-02, 1.18433734e-01,
       2.17128513e-01, 4.00082150e-01, 5.83035786e-01, 7.29807527e-01,
       8.76579267e-01, 1.02335101e+00, 1.18878795e+00, 1.35422489e+00,
       1.51966183e+00, 1.68509878e+00, 1.85801909e+00, 2.03093941e+00,
       2.20385972e+00, 2.37678004e+00, 2.54970035e+00, 2.72851002e+00,
       2.90731969e+00, 3.08612936e+00, 3.26493903e+00, 3.44374870e+00,
       3.62255837e+00, 3.79749514e+00, 3.97243190e+00, 4.14736867e+00,
       4.32230543e+00, 4.49724220e+00, 4.69082639e+00, 4.88441059e+00,
       5.07799478e+00, 5.27157898e+00, 5.52726271e+00, 5.78294644e+00,
       6.03863017e+00, 6.29431391e+00, 6.54999764e+00, 6.97267819e+00,
       7.39535873e+00, 7.81803928e+00, 8.24071983e+00, 8.81708624e+00,
       9.39345265e+00, 9.96981907e+00, 

4) Consider the node average, $\overline{x}(t) = \sum_{j=1}^N x_i(t)$. Compute the node average and plot the node average vs. time.  

In [ ]:
#Add code here


5) On Thursday, we will discuss the degree-based approximation which predicts that the disease will initially spread exponentially with the foloowing trend: $exp(t/\tau)$ with $\tau^{-1}  = \beta\left(\overline{k^2}/\overline{k}-1 \right)$. Code has been provided below to compute $\tau$. 
Does your computed node average follow this trend? Make a plot comparing your numerical result and this theoretical estimate. Note that we are interested in the rate of increase and not the magnitude of the node average.

In [ ]:
k = np.array(list(nx.degree(G)))[:,1]
kbar = k.mean()
k2bar = np.mean(k**2)
tau_inv = beta*(k2bar/kbar-1)
tau = 1/tau_inv

#Add code here


6) We can also estimate the rate of spread construting a linear least-squares fit of the form $log(\overline{x}) = at + b$ where $a$ and $b$ are the fit parameters. In the cell below, select ind1 and ind2, and then the code will construct such a fit for xbar[ind1:ind2] (assuming you have stored the node average in a variable named xbar). Is $a$ close to $1/\tau$?

In [ ]:
#ind1 = ?
#ind2 = ?
a,b = np.polyfit(t[ind1:ind2],np.log(xbar[ind1:ind2]),1)
print("a=",a)

**Note on solve_ivp:** *solve_ivp* generates approximate numerical solutions for systems of ODEs using one of several available variable time step methods. The solutions are computed moving forward from the initial condition one time step at a time. Each iteration, the length of the time step is set so that an estimated error falls below a given threshold (smaller time steps tend to correspond to more accurate solutions). We can assume that all of the available methods have been implemented efficiently, and there is no one method that works best for all IVPs. For systems of nonlinear ODEs, the BDF method is usually a good choice.